In [ ]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/cifar-10/"
DATA_WORKING = DATA_HOME_DIR# + "sample/"
DATA_WORKING_RESULTS = DATA_WORKING + "results/"

print ("DATA_WORKING      :", DATA_WORKING)
print ("DATA_WORKING_RESULTS:", DATA_WORKING_RESULTS)

In [ ]:
from keras import backend as K
from keras.models import load_model 
import os

print("----------------")
def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

In [ ]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [ ]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import pandas as pd
import scipy
from random import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import cPickle as pickle


In [ ]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

from vgg16 import Vgg16

In [ ]:
#path = HOMEPATH + "data/dogscats/sample/"
path = DATA_HOME_DIR 
#path = DATA_HOME_DIR + "sample/"
train_path = path + 'train/'
test_path = path + 'test/'
testUnknown_path = path + 'test/unknown/'
valid_path = path + 'valid/'
model_path = path + 'models/'
sample_path = path + 'sample/'
sampleTrain_path = path + 'sample/train/'
sampleTest_path = path + 'sample/test/'
sampleValid_path = path + 'sample/valid/'
sampleResults_path = path + 'sample/results/'
sampleTestUnknown_path = path + 'sample/test/unknown/'

categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


batch_size=64

In [ ]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]


In [ ]:
os.chdir(DATA_HOME_DIR)
!tree -d
#should display an ascii tree representation of directories.

In [ ]:
dirList = [train_path, valid_path, sampleTrain_path, sampleValid_path]

for dir_path in dirList:
    for category in categories:
        print ("# of files in", dir_path+category, ":", len(dirFileList(dir_path+category)))


In [ ]:
vgg = Vgg16()
model = vgg.model
print ("type(vgg):", type(vgg))
print ("type(model):", type(model))

In [ ]:
def showLayersInfo(model):
    print ("Number of layers : ", len(model.layers))
    for layer in model.layers:
        print (type(layer))

In [ ]:
showLayersInfo(model)

In [ ]:
model.summary()

In [ ]:
print("DATA_WORKING:", DATA_WORKING)

In [ ]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
startTime= datetime.now()
print ("startTime:", startTime)
#nb: get_batches is from utils.get_batches
val_batches = get_batches(DATA_WORKING+'valid/', shuffle=False, batch_size=1)
timeElapsed=datetime.now()-startTime
print('val_batches: Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))

print ("\n")

startTime= datetime.now()
print ("startTime:", startTime)
batches     = get_batches(DATA_WORKING+'train/', shuffle=False, batch_size=1)
timeElapsed=datetime.now()-startTime
print('batches : Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("val_batches:", type(val_batches))
print ("batches:", type(batches))

In [ ]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname):
    print ("load_array:fname:", fname)
    return bcolz.open(fname)[:]

In [ ]:
print ("DATA_WORKING:", DATA_WORKING)
#nb: get_data is from utils.get_data
startTime= datetime.now()
print ("startTime:", startTime)

val_data = get_data(DATA_WORKING+'valid')

print ("val_data:", type(val_data), val_data.shape)

timeElapsed=datetime.now()-startTime
print('val_data:Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("DATA_WORKING:", DATA_WORKING)
startTime= datetime.now()
print ("startTime:", startTime)

trn_data = get_data(DATA_WORKING+'train')
print ("trn_data:", type(trn_data), trn_data.shape)

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("DATA_WORKING_RESULTS:", DATA_WORKING_RESULTS)

In [ ]:
#utils.save_array uses bcolz. very small filesize generated.
save_array(DATA_WORKING_RESULTS+'train_data_full.bc', trn_data)
save_array(DATA_WORKING_RESULTS+'valid_data_full.bc', val_data)

In [ ]:
print ("DATA_WORKING_RESULTS:", DATA_WORKING_RESULTS)
print (DATA_WORKING_RESULTS+'train_data.bc')
#trn_data = load_array(DATA_WORKING_RESULTS+'train_data_full.bc')
#val_data = load_array(DATA_WORKING_RESULTS+'valid_data_full.bc')
print ("trn_data:", type(trn_data), trn_data.shape)#should return trn_data: <type 'numpy.ndarray'> (8000, 3, 224, 224)
print ("val_data:", type(val_data), val_data.shape)#should return val_data: <type 'numpy.ndarray'> (2000, 3, 224, 224)


In [ ]:
#Keras returns *classes* as a single column, so we convert to one hot encoding
def onehot(x): 
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [ ]:
val_classes = val_batches.classes
trn_classes = batches.classes
print ("val_classes:", type(val_classes), np.unique(val_classes), val_classes.shape)
print("val_classes[0:10]:", val_classes[0:10])
print ("trn_classes:", type(trn_classes), np.unique(val_classes), trn_classes.shape)
print("trn_classes[0:10]:", trn_classes[0:10])
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)
print ("after onehot conversion.")
print ("val_labels:", val_labels[0])
print ("trn_labels:", trn_labels[0])

In [ ]:
print ("start")
startTime= datetime.now()
print ("startTime:", startTime)

trn_features = model.predict(trn_data, batch_size=batch_size)

timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

val_features = model.predict(val_data, batch_size=batch_size)

timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("trn_features:", trn_features.shape)
print ("val_features:", val_features.shape)

In [ ]:
print ("DATA_WORKING_RESULTS:",DATA_WORKING_RESULTS)

In [ ]:
save_array(DATA_WORKING_RESULTS+'train_lastlayer_features_full.bc', trn_features)
save_array(DATA_WORKING_RESULTS+'valid_lastlayer_features_full.bc', val_features)

In [ ]:
#trn_features = load_array(DATA_WORKING_RESULTS+'train_lastlayer_features.bc')
#val_features = load_array(DATA_WORKING_RESULTS+'valid_lastlayer_features.bc')

In [ ]:
print ("trn_features:", type(trn_features), trn_features.shape)
print ("val_features:", type(val_features), val_features.shape)

In [ ]:
# 1000 inputs, since trn_features.shape[1]=1000, and 10 outputs, for # of categories
startTime= datetime.now()
print ("startTime:", startTime)


lm = Sequential([ Dense(10, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("lm:", type(lm))

In [ ]:
batch_size=64

In [ ]:
print (type(categories), len(categories))

In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print (type(lm))
print (lm.summary())

In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

preds = lm.predict_classes(val_features, batch_size=batch_size)

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))

In [ ]:
print (type(preds), preds.shape)
print ("preds[0:10]:", preds[0:10])

In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

#probabilities of each category results in numpy array with column for each category.
probs = lm.predict_proba(val_features, batch_size=batch_size)#[:,0]

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))

print ("probs:", type(probs), probs.shape)
print (probs[0,])
print ( ["%.4f" % v for v in probs[0,]])

In [ ]:
filenames = val_batches.filenames
print ("filenames:", type(filenames), len(filenames))
print (filenames[0:10])

In [ ]:
# Number of images to view for each visualization task
n_view = 4


In [ ]:
def classesFromFilenames(filenames):
    #returns text versions of categories from filenames.
    #requires filenames in format of 'category-text-form/123456.png'
    #strips out the category-text-form prior to '/'
    classes = []
    #print ("classesFromFilenames:idx:", idx)
    for i in range(0, len(filenames)):
        pos = filenames[i].find('/')
        #print ("pos:", pos, filenames[i][0:pos])
        classes.append(filenames[i][0:pos])
    return classes

In [ ]:
def classesFromPreds(preds, categories):
    #returns text versions of categories
    categs = []
    for pred in preds:
        #print("pred:", pred, categories[pred])
        categs.append(categories[pred])
    return categs

In [ ]:
def plots_idx(path, idx, titles=None):
    #titles = numpy array idx rows x 10 columns, need to select highest prob from each row.
    maxProb = []
    for i in range(0, titles.shape[0]):
        #print (titles[i,], type(titles[i,]))
        #print (np.argmax(titles[i,]))
        maxProb.append(titles[i,np.argmax(titles[i,])])
    plots([image.load_img(path + filenames[i]) for i in idx], titles=maxProb)

In [ ]:
categsFromFileNames = classesFromFilenames(filenames)
print("categsFromFileNames:", len(categsFromFileNames), categsFromFileNames[0:4])
categsFromPreds = classesFromPreds(preds, categories)
print("categsFromPreds:", len(categsFromPreds), categsFromPreds[0:4])


In [ ]:
def plotSamples(matchIndex):
    #numpy.random.permutation
    idx = permutation(matchIndex)[:n_view]
    print ("idx:", idx)
    for i in idx:
        print(filenames[i])

    #print (classesFromFilenames(idx, filenames))
    #print ("probs[idx].shape:", probs[idx].shape)
    plots_idx(DATA_WORKING+'valid/', idx, probs[idx])
    #print ("idx:", idx)
    print ("actual from filenames:", [categsFromFileNames[i] for i in idx])
    print ("predicted            :",     [categsFromPreds[i] for i in idx])
    #print ("probs[idx[0],]:", probs[idx[0],])

In [ ]:
#select correctly predicted.
matchIndex = []
for i in range(0,len(categsFromFileNames)):
    if categsFromFileNames[i]==categsFromPreds[i]:
        matchIndex.append(i)
print ("# of correctly predicted : matchIndex:", len(matchIndex))
plotSamples(matchIndex)

In [ ]:
#select incorrectly predicted.
matchIndex = []
for i in range(0,len(categsFromFileNames)):
    if categsFromFileNames[i]!=categsFromPreds[i]:
        matchIndex.append(i)
print ("# of incorrectly predicted : matchIndex:", len(matchIndex))
plotSamples(matchIndex)

In [ ]:
cm = confusion_matrix(val_classes, preds)

In [ ]:
print ("type(val_batches.class_indices):", type(val_batches.class_indices))

In [ ]:
#plt.figure(figsize=(12,12)) #does not affect size, method is inside utils

plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
vgg.model.summary()

In [ ]:
print ("before pop, # of layers:", len(model.layers))
if len(model.layers)==38:
    print ("# of layers == 38., poping top layer.")
    model.pop()
else:
    print ("# of layers not == 38.")
print ("after pop, # of layers:", len(model.layers))
for layer in model.layers: layer.trainable=False

In [ ]:
print (type(vgg.model))
#not really useful, here for reference only
#for layer in model.layers:
#    print(layer.name, layer.inbound_nodes, layer.outbound_nodes)

In [ ]:
vgg.model.summary()

In [ ]:
model.add(Dense(10, activation='softmax'))#NB: Dense requires 10 because 10 categories.

In [ ]:
print ("after adding dense layer, # of layers:", len(model.layers))

In [ ]:
vgg.model.summary()

In [ ]:
gen=image.ImageDataGenerator()
print ("gen:", type(gen))
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
#https://keras.io/preprocessing/image/
#flow(x, y): Takes numpy data & label arrays, 
#and generates batches of augmented/normalized data. 
#Yields batches indefinitely, in an infinite loop.

val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

print ("batches:", type(batches))
print ("val_batches:", type(val_batches))

In [ ]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, 
                        samples_per_epoch=batches.n, 
                        nb_epoch=nb_epoch, 
                        validation_data=val_batches, 
                        nb_val_samples=val_batches.n)

In [ ]:
opt = RMSprop(lr=0.1)
#https://keras.io/optimizers/#rmsprop
#This optimizer is usually a good choice for recurrent neural networks.
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
startTime= datetime.now()
fit_model(model, batches, val_batches, nb_epoch=2)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

In [ ]:
print ("DATA_WORKING_RESULTS:", DATA_WORKING_RESULTS)

In [ ]:
model.save_weights(DATA_WORKING_RESULTS+'finetune1_full.h5')

In [ ]:
#model.load_weights(DATA_WORKING_RESULTS+'finetune1.h5')

In [ ]:
model.evaluate(val_data, val_labels)

In [ ]:
print ("type(model):", type(model))

In [ ]:
preds = model.predict_classes(val_data, batch_size=batch_size)
probs = model.predict_proba(val_data, batch_size=batch_size)

In [ ]:
print("preds:", preds.shape)
print("probs:", probs.shape)

In [ ]:
probs[:8,]

In [ ]:
cm = confusion_matrix(val_classes, preds)

In [ ]:
print ("type(val_batches):", type(val_batches))

In [ ]:
#recall - val_batches was generated from loading files. possibly need to initialize again?
#val_batches = get_batches(DATA_WORKING+'valid/', shuffle=False, batch_size=1)
#plot_confusion_matrix(cm, val_batches.class_indices)
#AttributeError: 'NumpyArrayIterator' object has no attribute 'class_indices'

In [ ]:
for layer in model.layers:
    print (type(layer))

In [ ]:
layers = model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
print (type(first_dense_idx), first_dense_idx)
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: 
    layer.trainable=True

In [ ]:
type(K)
#recall from above: from keras import backend as K

In [ ]:
K.set_value(opt.lr, 0.01)
#https://keras.io/backend/
#keras.backend.set_value(x, value)
#https://www.tensorflow.org/api_docs/python/tf/keras/backend/set_value

startTime= datetime.now()
print ("startTime:", startTime)

fit_model(model, batches, val_batches, 3)

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
model.save_weights(DATA_WORKING_RESULTS+'finetune2_.h5')

In [ ]:
for layer in layers[12:]: layer.trainable=True
K.set_value(opt.lr, 0.001)

In [ ]:
fit_model(model, batches, val_batches, 4)

In [ ]:
model.save_weights(DATA_WORKING_RESULTS+'finetune3_full.h5')

In [ ]:
from keras.models import load_model #move this to top when run complete.
model.save(DATA_WORKING_RESULTS+'CIFAR-10_part2_final_model_full.h5')

In [ ]:
#this did not work.
def save_object(obj, filename):
    filename = DATA_WORKING_RESULTS+filename
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    print ("method save_object loaded with DATA_WORKING_RESULTS=", DATA_WORKING_RESULTS)
# sample usage
#save_object(company1, 'company1.pkl')

In [ ]:
#save_object(model, 'CIFAR-10_part2_final_model.pkl')

In [ ]:
#pickle_model = pickle.load( open( DATA_WORKING_RESULTS+'CIFAR-10_part2_final_model.pkl', "rb" ) )

In [ ]:
#print(type(pickle_model))

In [ ]:
val_features = load_array(DATA_WORKING_RESULTS+'valid_lastlayer_features.bc')
print ("val_features:", type(val_features), val_features.shape)

In [ ]:
model.summary()

In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

preds = model.predict_classes(val_features, batch_size=batch_size)

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))